In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Pre-trained ResNet-50 Model Setup
class ResNet50FeatureExtractor(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet50FeatureExtractor, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Remove the classification layer
        self.classifier = nn.Linear(resnet.fc.in_features, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x).flatten(1)
        out = self.classifier(features)
        return out, features

In [4]:
# Entropy Calculation
def calculate_entropy(predictions):
    """
    Calculate entropy for a batch of predictions.
    """
    probabilities = F.softmax(predictions, dim=1)
    entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-5), dim=1)
    return entropy


# Threshold Definition
def define_entropy_threshold(entropy_values, alpha=0.5, beta=0.5):
    """
    Define dynamic threshold using mean and standard deviation.
    T = alpha * mean + beta * std
    """
    mean_entropy = torch.mean(entropy_values)
    std_entropy = torch.std(entropy_values)
    threshold = alpha * mean_entropy + beta * std_entropy
    return threshold


# Separate Samples into High-Entropy and Low-Entropy Groups
def separate_samples(entropy_values, threshold):
    """
    Split samples into high-entropy and low-entropy groups.
    """
    low_entropy_indices = (entropy_values < threshold).nonzero(as_tuple=True)[0]
    high_entropy_indices = (entropy_values >= threshold).nonzero(as_tuple=True)[0]
    return low_entropy_indices, high_entropy_indices

In [5]:
def pipeline():
    pass

In [6]:
# MNIST Target Dataset Setup
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.Resize((224, 224)),  # Resize to ResNet-50 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize to [-1, 1]
])

# Load MNIST as Target Domain
mnist_target = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
target_loader = DataLoader(mnist_target, batch_size=16, shuffle=False)

# Run the pipeline
low_entropy_indices, high_entropy_indices, features = pipeline(target_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 164MB/s]


KeyboardInterrupt: 